In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import TimestampType
import pyspark.sql.functions as F
import pandas as pd
import plotly.express as px

spark = SparkSession.builder.getOrCreate()

22/08/16 09:59:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/08/16 09:59:02 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/08/16 09:59:02 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/08/16 09:59:02 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
22/08/16 09:59:02 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
22/08/16 09:59:02 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.
22/08/16 09:59:02 WARN Utils: Service 'SparkUI' could not bind on port 4045. Attempting port 4046.
22/08/16 09:59:02 WARN Utils: Service 'SparkUI' could

In [2]:
# 6
olist_orders_dataset = spark.read.options(header='True', inferSchema='True', delimiter=',') \
                            .csv("data/olist_orders_dataset.csv")

In [3]:
olist_orders_dataset.columns

['order_id',
 'customer_id',
 'order_status',
 'order_purchase_timestamp',
 'order_approved_at',
 'order_delivered_carrier_date',
 'order_delivered_customer_date',
 'order_estimated_delivery_date']

In [4]:
orders = olist_orders_dataset.select("order_id","order_delivered_carrier_date","order_estimated_delivery_date")
orders = orders.withColumn('order_delivered_carrier_date',to_date(unix_timestamp \
            (orders.order_delivered_carrier_date, 'yyyy-MM-dd HH:mm:ss').cast('timestamp'))) \
            .withColumn('order_estimated_delivery_date',to_date(unix_timestamp \
            (orders.order_estimated_delivery_date, 'yyyy-MM-dd HH:mm:ss').cast('timestamp')))
orders.show()

+--------------------+----------------------------+-----------------------------+
|            order_id|order_delivered_carrier_date|order_estimated_delivery_date|
+--------------------+----------------------------+-----------------------------+
|e481f51cbdc54678b...|                  2017-10-04|                   2017-10-18|
|53cdb2fc8bc7dce0b...|                  2018-07-26|                   2018-08-13|
|47770eb9100c2d0c4...|                  2018-08-08|                   2018-09-04|
|949d5b44dbf5de918...|                  2017-11-22|                   2017-12-15|
|ad21c59c0840e6cb8...|                  2018-02-14|                   2018-02-26|
|a4591c265e18cb1dc...|                  2017-07-11|                   2017-08-01|
|136cce7faa42fdb2c...|                        null|                   2017-05-09|
|6514b8ad8028c9f2c...|                  2017-05-22|                   2017-06-07|
|76c6e866289321a7c...|                  2017-01-26|                   2017-03-06|
|e69bfb5eb88e0ed

In [5]:
orders.dtypes

[('order_id', 'string'),
 ('order_delivered_carrier_date', 'date'),
 ('order_estimated_delivery_date', 'date')]

#### We drop rows with null values because we assume that these orders never will by deliverd

In [6]:
orders = orders.na.drop("any")

#### two_days column description
__True__ - order was sent more then 2 days before expected delivery date   
__False__ - The order has not been shipped more than two days before the expected delivery date

In [7]:
orders_big = orders.withColumn('two_days', F.date_add(orders['order_delivered_carrier_date'],2))
orders_big = orders_big.withColumn("two_days", F.when(F.col("two_days") < F.col("order_estimated_delivery_date"), True) \
                                   .otherwise(False))
orders_big.show()

+--------------------+----------------------------+-----------------------------+--------+
|            order_id|order_delivered_carrier_date|order_estimated_delivery_date|two_days|
+--------------------+----------------------------+-----------------------------+--------+
|e481f51cbdc54678b...|                  2017-10-04|                   2017-10-18|    true|
|53cdb2fc8bc7dce0b...|                  2018-07-26|                   2018-08-13|    true|
|47770eb9100c2d0c4...|                  2018-08-08|                   2018-09-04|    true|
|949d5b44dbf5de918...|                  2017-11-22|                   2017-12-15|    true|
|ad21c59c0840e6cb8...|                  2018-02-14|                   2018-02-26|    true|
|a4591c265e18cb1dc...|                  2017-07-11|                   2017-08-01|    true|
|6514b8ad8028c9f2c...|                  2017-05-22|                   2017-06-07|    true|
|76c6e866289321a7c...|                  2017-01-26|                   2017-03-06|    true|

In [8]:
orders_big.dtypes

[('order_id', 'string'),
 ('order_delivered_carrier_date', 'date'),
 ('order_estimated_delivery_date', 'date'),
 ('two_days', 'boolean')]

In [ ]:
pie = {}
pie["t_value"] = orders_big.where(F.col('two_days')=="true").count()
pie["f_value"] = orders_big.where(F.col('two_days')=="false").count()
df = pd.DataFrame(pie.items())

In [ ]:
fig = px.pie(df, values=1, 
             names=["Shipped more than 2 days before scheduled delivery date","Shipped less than 2 days before scheduled delivery date"],
             title='Percentage of parcels not shipped more than 2 days before the scheduled delivery date')
fig.show()